In [11]:
#Importing Libraries
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import keras as k
import seaborn as sns                     # data visualization library based on matplotlib
import sklearn                            # for random forests, and k-neighbours etc 

In [2]:
data=pd.read_csv('anemia.csv')
data.head()

,Gender,Hemoglobin,MCH,MCHC,MCV,Result
0,1,14.9,22.7,29.1,83.7,0
1,0,15.9,25.4,28.3,72.0,0
2,0,9.0,21.5,29.6,71.2,1
3,0,14.9,16.0,31.4,87.5,0
4,1,14.7,22.0,28.2,99.5,0


In [3]:
print(data.shape)

(1421, 6)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1421 entries, 0 to 1420
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      1421 non-null   int64  
 1   Hemoglobin  1421 non-null   float64
 2   MCH         1421 non-null   float64
 3   MCHC        1421 non-null   float64
 4   MCV         1421 non-null   float64
 5   Result      1421 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 66.7 KB


In [5]:
data.isnull().sum()

Gender        0
Hemoglobin    0
MCH           0
MCHC          0
MCV           0
Result        0
dtype: int64

In [6]:
data.describe()

,Gender,Hemoglobin,MCH,MCHC,MCV,Result
count,1421.000000,1421.000000,1421.000000,1421.000000,1421.000000,1421.000000
mean,0.520760,13.412738,22.905630,30.251232,85.523786,0.436312
std,0.499745,1.974546,3.969375,1.400898,9.636701,0.496102
min,0.000000,6.600000,16.000000,27.800000,69.400000,0.000000
25%,0.000000,11.700000,19.400000,29.000000,77.300000,0.000000
50%,1.000000,13.200000,22.700000,30.400000,85.300000,0.000000
75%,1.000000,15.000000,26.200000,31.400000,94.200000,1.000000
max,1.000000,16.900000,30.000000,32.500000,101.600000,1.000000


In [7]:
data['Result'].value_counts()

0    801
1    620
Name: Result, dtype: int64

In [9]:
data.groupby('Result').mean()

,Gender,Hemoglobin,MCH,MCHC,MCV
Result,,,,,
0,0.409488,14.795506,23.005743,30.192010,85.698127
1,0.664516,11.626290,22.776290,30.327742,85.298548


In [14]:
X = data.drop(columns=['Result'], axis=1)
Y = data['Result']   

In [15]:
#Feature Scaling 
#Min-Max scaler method scales the dataset so that all the input features lie between 0 and 1
X_scaler = MinMaxScaler()
X_scaler.fit(X)
column_names = X.columns
X[column_names] = X_scaler.transform(X)

In [16]:
print(X)

      Gender  Hemoglobin       MCH      MCHC       MCV
0        1.0    0.805825  0.478571  0.276596  0.444099
1        0.0    0.902913  0.671429  0.106383  0.080745
2        0.0    0.233010  0.392857  0.382979  0.055901
3        0.0    0.805825  0.000000  0.765957  0.562112
4        1.0    0.786408  0.428571  0.085106  0.934783
...      ...         ...       ...       ...       ...
1416     0.0    0.388350  0.671429  0.085106  0.419255
1417     1.0    0.533981  0.878571  0.553191  0.543478
1418     1.0    0.631068  0.121429  0.063830  0.350932
1419     0.0    0.747573  0.014286  0.361702  0.801242
1420     0.0    0.504854  0.371429  0.127660  0.891304

[1421 rows x 5 columns]


In [17]:
print(Y)

0       0
1       0
2       1
3       0
4       0
       ..
1416    1
1417    1
1418    1
1419    0
1420    1
Name: Result, Length: 1421, dtype: int64


In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [20]:
print(X.shape, X_train.shape, X_test.shape)

(1421, 5) (1136, 5) (285, 5)


In [21]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [22]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
print(X_train)

[[-1.02495928 -0.01991223  1.258941   -0.89951627 -0.09305298]
 [-1.02495928 -1.29071532  1.43583429 -1.18938238 -0.05153366]
 [ 0.97564852 -1.18905107 -0.20674626 -0.60965017  1.21480561]
 ...
 [-1.02495928 -2.15486142 -0.91431942  0.83968036 -0.66394364]
 [-1.02495928  1.75921209  0.65244972 -0.46471712 -0.21761094]
 [ 0.97564852  0.08175202 -0.18147579 -0.46471712  1.1421468 ]]


In [24]:
#Build the model
model = Sequential()
model.add( Dense(256, input_dim = len(X.columns), kernel_initializer= k.initializers.random_normal(seed=13), activation='relu'))
model.add( Dense(1, activation = 'relu'))

In [25]:
#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
#Train the model
history = model.fit(X_train, Y_train, epochs = 500, batch_size = X_train.shape[0])

Epoch 1/500
1/1 [==============================] - 0s 17ms/step - loss: 0.4046 - accuracy: 0.8539
Epoch 2/500
1/1 [==============================] - 0s 15ms/step - loss: 0.4002 - accuracy: 0.8565
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 0.3960 - accuracy: 0.8574
Epoch 4/500
1/1 [==============================] - 0s 16ms/step - loss: 0.3918 - accuracy: 0.8583
Epoch 5/500
1/1 [==============================] - 0s 15ms/step - loss: 0.3877 - accuracy: 0.8556
Epoch 6/500
1/1 [==============================] - 0s 14ms/step - loss: 0.3836 - accuracy: 0.8592
Epoch 7/500
1/1 [==============================] - 0s 15ms/step - loss: 0.3794 - accuracy: 0.8618
Epoch 8/500
1/1 [==============================] - 0s 21ms/step - loss: 0.3754 - accuracy: 0.8627
Epoch 9/500
1/1 [==============================] - 0s 17ms/step - loss: 0.3714 - accuracy: 0.8680
Epoch 10/500
1/1 [==============================] - 0s 17ms/step - loss: 0.3674 - accuracy: 0.8680
Epoch 11/500
1/1 [=

In [31]:
input_data = (1,8.4,20.1,30,75.6)

# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the data
std_data = scaler.transform(input_data_reshaped)

prediction = model.predict(std_data)
print(prediction)


if (prediction[0]==0):
  print("The Person has Anaemia")

else:
  print("The Person is healthy")

1/1 [==============================] - 0s 54ms/step
[[0.]]
The Person has Anaemia


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
